In [1]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
region = 13
mother_directory = f"/content/drive/MyDrive/MS thesis/[optimize] Training, Testing/RG{region}/"
# raw_rg1_clustered = pd.read_csv("/content/drive/MyDrive/MS thesis/preliminary site selection/CALABARZON/calabarzon_clusters.csv")
raw_rg1_clustered = pd.read_csv(mother_directory + "PCF_data.csv")
# use pcf data nalang para you dont get nans for sure?

neighbors_df = pd.read_csv(mother_directory + 'neighbours.csv')
candidate_sites = pd.read_csv(mother_directory + "candidate_sites.csv")
candidate_sites

'''AGGREGATING THE POPULATION'''
# Assuming df is your DataFrame
# vp =  raw_rg1_clustered['popden_chi'] + raw_rg1_clustered['popden_eld'] + raw_rg1_clustered['popden_wom'] + raw_rg1_clustered['popden_you'] + raw_rg1_clustered['popden_w_1'] + raw_rg1_clustered['popden_you']
# gen = raw_rg1_clustered['popden_all'] - vp
# all = vp + gen
# raw_rg1_clustered['total_population'] = all
# raw_rg1_clustered.columns

# raw_rg1_clustered = raw_rg1_clustered.drop(columns=['Unnamed: 0'])
neighbors_df.rename(columns={'fid': 'ID'}, inplace=True)
# raw_rg1_clustered = raw_rg1_clustered.drop(columns=['HCFAI.1'])
raw_rg1_clustered.columns

Index(['popden_chi', 'popden_eld', 'popden_wom', 'popden_you', 'popden_w_1',
       'popden_all', 'flood_probability_value', 'rain intensity_value',
       'drought_value', 'Distance_to_Nearest_RHU_km', 'HCFAI',
       'total_population', 'RHU_Presence', 'ID', 'buildability_landcov',
       'Road_Presence', 'POI_Presence', 'Nearest_RHU', 'Cluster'],
      dtype='object')

In [3]:
# Define the HCI calculation function
def HCI_calc(total_ai, total_gi, total_hi, total_ji, total_ki, total_mi,distance, road_bi, POI_ci, landCov_di, hazard1_ei, hazard2_ei, hazard3_ei, rhus_fi):
    total_vulnerable = total_gi + total_hi + total_ji + total_ki + total_mi
    total_pop = total_ai
    population_to_be_served = total_vulnerable + np.maximum(0, total_pop - total_vulnerable)
    # Calculate y for the entire Series without using if condition
    y = np.where(population_to_be_served == 0, 0, 20000 / ((population_to_be_served) * (distance + rhus_fi)))
    mc = np.tanh(y)
    w_bi = 0.3 # roads
    w_ci = 0.2 # POIs
    w_di = 0.5 # land cov
    b = (POI_ci * w_ci) + (road_bi * w_bi) + (landCov_di * w_di)
    # Normalize each factor
    rain_intensity_normalized = (hazard1_ei - hazard1_ei.min()) / (hazard1_ei.max() - hazard1_ei.min())
    flood_probability_normalized = (hazard2_ei - hazard2_ei.min()) / (hazard2_ei.max() - hazard2_ei.min())
    drought_mean_normalized = (hazard3_ei - hazard3_ei.min()) / (hazard3_ei.max() - hazard3_ei.min())
    w_rain = 0.4
    w_flood = 0.3
    w_drought = 0.3
    c = (w_rain * rain_intensity_normalized) + (w_flood * flood_probability_normalized) + (w_drought * drought_mean_normalized)
    f = b - c
    f = np.tanh(f)
    hci = mc * f
    hcfai = (1 + np.tanh(hci / 2)) / 2  # Sigmoid function
    return hcfai

# Computing overall HCFAI using SUM
def overallHCFAI(region_df):
    HCFAI_overall = region_df['HCFAI'].sum()
    return HCFAI_overall

# Select top N candidate sites based on HCFAI score computed from region_df
def selectTopSites(candidate_sites, region_df, n):
    # Merge candidate_sites with region_df based on a common identifier (e.g., ID)
    # print(f"Region df: {region_df.columns}")
    # print(f"Candidate sites: {candidate_sites.columns}")
    # Define the columns to merge
    # columns_to_merge = ['ID', 'barangay_name', 'city_name', 'province_name',
    #                     'popden_chi', 'popden_eld', 'popden_wom', 'popden_you', 'popden_w_1',
    #                     'popden_all', 'flood_probability_value', 'rain intensity_value',
    #                     'drought_value', 'buildability_landcov', 'RHU_Presence',
    #                     'Road_Presence', 'POI_Presence', 'Nearest_RHU',
    #                     'Distance_to_Nearest_RHU_km', 'total_population']
    columns_to_merge = ['ID',
                        'popden_chi', 'popden_eld', 'popden_wom', 'popden_you', 'popden_w_1',
                        'popden_all', 'flood_probability_value', 'rain intensity_value',
                        'drought_value', 'buildability_landcov', 'RHU_Presence',
                        'Road_Presence', 'POI_Presence', 'Nearest_RHU',
                        'Distance_to_Nearest_RHU_km', 'total_population']

    # Merge only the specified columns from region_df to candidate_sites
    merged_sites = candidate_sites.merge(region_df[columns_to_merge], on='ID', how='left')
    # print(f"Merged: {merged_sites.columns}")
    # Create a new DataFrame without the column 'Unnamed: 0'
    # merged_sites = candidate_sites.merge(region_df, on='ID', how='left')
    # Compute HCFAI scores for candidate sites using data from region_df
    merged_sites['HCFAI'] = HCI_calc(merged_sites['popden_all'],
                                      merged_sites['popden_chi'],
                                      merged_sites['popden_eld'],
                                      merged_sites['popden_wom'],
                                      merged_sites['popden_you'],
                                      merged_sites['popden_w_1'],
                                      merged_sites['Distance_to_Nearest_RHU_km'],
                                      merged_sites['Road_Presence'],
                                      merged_sites['POI_Presence'],
                                      merged_sites['buildability_landcov'],
                                      merged_sites['rain intensity_value'],
                                      merged_sites['flood_probability_value'],
                                      merged_sites['drought_value'],
                                      merged_sites['RHU_Presence'],
                                     )

    # Fill missing values from region_df to candidate_sites
    missing_cols = [col for col in region_df.columns if col != ['ID']]
    for col in missing_cols:
        # Set the index of region_df to 'ID' if it's not already set
        if 'ID' not in region_df.columns:
            region_df.set_index('ID', inplace=True)
        # Fill missing values in merged_sites using values from region_df
        merged_sites[col] = merged_sites[col].fillna(region_df[col])

    # Select top N sites based on computed HCFAI scores
    top_sites = merged_sites.sort_values(by='HCFAI', ascending=False).head(n)
    return top_sites

# Remove adjacent sites from the list of candidate sites
def removeAdjacentSites(region_df, candidate_sites, selected_sites, neighbors_df):
    # Get the indices of hexagons with existing health facilities
    idx_with_RHU = region_df[region_df['RHU_Presence'] == 1]
    # Initialize a set to store adjacent sites
    adjacent_sites = set()
    # Add neighbors of selected sites
    for site_id in selected_sites['ID']:
      if site_id in neighbors_df['ID'].values:
          neighbors = neighbors_df.loc[neighbors_df['ID'] == site_id, 'neighbours'].iloc[0]
          adjacent_sites.update(neighbors.split(','))
    # Add hexagons with existing health facilities and the selected sites themselves
    adjacent_sites.update(idx_with_RHU['ID'])
    adjacent_sites.update(selected_sites['ID'])
    # Convert the adjacent sites to integers
    adjacent_sites = [int(site) for site in adjacent_sites]
    # Remove adjacent sites from the candidate sites
    candidate_sites = candidate_sites[~candidate_sites['ID'].isin(adjacent_sites)]
    # Recalculate HCFAI values for selected sites using data from region_df
    selected_sites['HCFAI'] = HCI_calc(selected_sites['popden_all'], selected_sites['popden_chi'],
                                       selected_sites['popden_eld'], selected_sites['popden_wom'],
                                       selected_sites['popden_you'], selected_sites['popden_w_1'],
                                       selected_sites['Distance_to_Nearest_RHU_km'],
                                       selected_sites['Road_Presence'], selected_sites['POI_Presence'],
                                       selected_sites['buildability_landcov'], selected_sites['rain intensity_value'],
                                       selected_sites['flood_probability_value'], selected_sites['drought_value'],
                                       selected_sites['RHU_Presence'])
    return candidate_sites, selected_sites, idx_with_RHU

# Optimize the selection of health facility sites
def optimize(region_df, candidate_sites, neighbors_df, num_facilities):
  # Print original HCFAI
  original_HCFAI = overallHCFAI(region_df)
  print("Original HCFAI:", original_HCFAI)
  selected_sites = pd.DataFrame(columns=region_df.columns)
  while len(selected_sites) < num_facilities:
    # Select top candidate sites
    top_sites = selectTopSites(candidate_sites, region_df, num_facilities - len(selected_sites))
    # Remove adjacent sites from candidate list and update HCFAI for selected sites
    candidate_sites, top_sites, idx_with_RHU = removeAdjacentSites(region_df, candidate_sites, top_sites, neighbors_df)
    # Add selected sites to the final list
    selected_sites = pd.concat([selected_sites, top_sites])

    # Calculate HCFAI for the remaining sites in region_df
    remaining_sites = region_df[~region_df['ID'].isin(selected_sites['ID'])]
    remaining_HCFAI = overallHCFAI(remaining_sites)
    selected_sites_HCFAI = overallHCFAI(selected_sites)

    # Calculate updated overall HCFAI
    updated_HCFAI = remaining_HCFAI + selected_sites_HCFAI
    print("Updated HCFAI:", updated_HCFAI)

    if len(selected_sites) == num_facilities:
        print("Accept!!!!!!!!!!!!!!")
        return selected_sites, idx_with_RHU, original_HCFAI, updated_HCFAI
    else:
        print("Reject!!!!!!!!!!!!!!")
        print("Selected sites:", len(selected_sites))

  return selected_sites, idx_with_RHU, original_HCFAI, updated_HCFAI

In [10]:
print(f"Candidate sites: {len(candidate_sites)}")
# Optimize selection of health facility sites
selected_facilities, hex_with_RHU, og_HCFAI, updated_HCFAI = optimize(raw_rg1_clustered, candidate_sites, neighbors_df, 1)
print(selected_facilities.to_string(index=False))

Candidate sites: 2201
Original HCFAI: 1919.5258770785804
Updated HCFAI: 1918.9281232230178
Accept!!!!!!!!!!!!!!
 popden_chi  popden_eld  popden_wom  popden_you  popden_w_1  popden_all  flood_probability_value  rain intensity_value  drought_value  Distance_to_Nearest_RHU_km HCFAI  total_population RHU_Presence      ID buildability_landcov Road_Presence POI_Presence Nearest_RHU Cluster  pred_y  actual_y
        0.0         0.0         0.0         0.0         0.0    0.010407                 0.568073              0.022628       0.560882                    0.692577   NaN          0.010407            0 3460374                    1             0            0     3443757       0     1.0       0.0


In [11]:
selected_facilities = list(selected_facilities['ID'])
hex_with_RHU = list(hex_with_RHU['ID'])
hex_with_RHU
print(f"Optimal sites: {len(selected_facilities)}")
print(f"Sites with RHUs: {len(hex_with_RHU)}")

Optimal sites: 1
Sites with RHUs: 5


In [12]:
selected_facilities

[3460374]

In [13]:
hex_with_RHU

[3362142, 3398135, 3407800, 3432642, 3443757]

# Validation

In [14]:
import random
import pandas as pd

def getRandom48(candidate_sites, random_seed=None):
    """
    Randomly select 48 sites from the candidate sites DataFrame.

    Parameters:
        candidate_sites (DataFrame): DataFrame containing information about all candidate sites.
        random_seed (int): Random seed for reproducibility.

    Returns:
        DataFrame: Randomly selected 48 sites DataFrame.
    """
    if random_seed is not None:
        random.seed(random_seed)
    candidate_sites_shuffled = candidate_sites.sample(frac=1).reset_index(drop=True)
    random_48 = candidate_sites_shuffled.sample(n=48)
    return random_48


def randomize(region_df, candidate_sites, random_seed):
    """
    Randomize the selection of 48 sites and update the overall HCFAI value.
    Parameters:
        region_df (DataFrame): DataFrame containing information about all sites in the region.
    Returns:
        tuple: Tuple containing the updated overall HCFAI value and the IDs of randomly selected sites.
    """
    HCFAI_overall_before = overallHCFAI(region_df)
    # print("Overall HCFAI (before randomization): ", HCFAI_overall_before)
    random_48 = getRandom48(candidate_sites, random_seed)
    columns_to_merge = ['ID',
                        'popden_chi', 'popden_eld', 'popden_wom', 'popden_you', 'popden_w_1',
                        'popden_all', 'flood_probability_value', 'rain intensity_value',
                        'drought_value', 'buildability_landcov', 'RHU_Presence',
                        'Road_Presence', 'POI_Presence', 'Nearest_RHU',
                        'Distance_to_Nearest_RHU_km', 'HCFAI', 'total_population']
    merged_sites = random_48.merge(region_df[columns_to_merge], on='ID', how='left')
    missing_cols = [col for col in region_df.columns if col != ['ID']]
    for col in missing_cols:
        if 'ID' not in region_df.columns:
            region_df.set_index('ID', inplace=True)
        merged_sites[col] = merged_sites[col].fillna(region_df[col])
    # Calculate HCFAI for the remaining sites in region_df
    remaining_sites = region_df[~region_df['ID'].isin(merged_sites['ID'])]
    remaining_HCFAI = overallHCFAI(remaining_sites)
    print(remaining_HCFAI)
    selected_sites_HCFAI = overallHCFAI(merged_sites)
    print(selected_sites_HCFAI)

    # Calculate updated overall HCFAI
    updated_HCFAI = remaining_HCFAI + selected_sites_HCFAI
    # print(f"Updated HCFAI: ", updated_HCFAI)
    return updated_HCFAI, random_48['ID'].tolist()

# Example usage
rdm_hcfai = []
rdm_points = []
for i in range(10):
    random_seed = i  # Use a different random seed for each iteration
    rdm_ovHCFAI, rdm_48 = randomize(raw_rg1_clustered, candidate_sites, random_seed)
    rdm_hcfai.append(rdm_ovHCFAI)
    rdm_points.append(rdm_48)

rdm_HCFAI_avg = sum(rdm_hcfai) / len(rdm_hcfai)
highest_rdm_HCFAI = max(rdm_hcfai)
# print("Average HCFAI after randomization: ", rdm_HCFAI_avg)

Results_rdmsum = pd.DataFrame({
    'Hex IDs': rdm_points,
    'HCFAI': rdm_hcfai
})

print("=========== OG HCFAI RESULTS ===========")
print("ORIGINAL HCFAI", og_HCFAI)
print("=========== RANDOM HCFAI RESULTS ===========")
# print("RANDOMIZED HCFAI [All, 10-fold]", rdm_hcfai)
print("RANDOMIZED HCFAI [10-fold]: ", highest_rdm_HCFAI)
# print("RANDOMIZED HCFAI AVG[10 fold]: ", rdm_HCFAI_avg)
print("=========== OPTIMIZED HCFAI RESULTS ===========")
print("OPTIMIZED HCFAI ", updated_HCFAI)

1896.1217526894675
23.404124389113285
1896.018379272759
23.507497805821526
1896.4250421361312
23.100834942449218
1897.997150938867
21.528726139713573
1897.2473778269182
22.278499251662318
1897.2700869968717
22.255790081708838
1896.7358251885646
22.790051890015675
1897.2519269017366
22.27395017684389
1896.5105568075637
23.015320271016495
1896.7975912779025
22.72828580067793
=========== OG HCFAI RESULTS ===========
ORIGINAL HCFAI 1919.5258770785804
=========== RANDOM HCFAI RESULTS ===========
RANDOMIZED HCFAI [10-fold]:  1919.5258770785808
=========== OPTIMIZED HCFAI RESULTS ===========
OPTIMIZED HCFAI  1918.9281232230178


In [15]:
hcfai_compare_df =  pd.DataFrame(
    {' ': ['SUM'],
    'Old HCFAI': og_HCFAI,
     'Randomized HCFAI': highest_rdm_HCFAI,
     'Algorithm HCFAI': updated_HCFAI
    })
hcfai_compare_df

,,Old HCFAI,Randomized HCFAI,Algorithm HCFAI
0,SUM,1919.525877,1919.525877,1918.928123
